In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import networkx as nx
from matching import read_sheet, DAYS, SHIFTS

In [ ]:
df, shift_type_dict = read_sheet("Shiftsplan_lux025_3.xlsx")
df

In [ ]:
shift_degree = [df[s].sum() for s in shift_type_dict]
len([x for x in shift_degree if x < 2])
for i,key in enumerate(shift_type_dict):
    if shift_degree[i] < 2:
        shift_type_dict[key] = []

In [ ]:
node_lengths = [1,
                df.shape[0],
                (df.shape[0] * len(SHIFTS) * len(DAYS)),
                (len(SHIFTS) * len(DAYS)),
                (len(SHIFTS) * len(DAYS)),
                (len(SHIFTS) * len(DAYS)),
                1]
node_offsets = [reduce(lambda x,y: x+y, node_lengths[:i]) for i in range(1,len(node_lengths))]
print(node_offsets)
n_nodes = reduce(lambda x,y: x+y, node_lengths)
graph_matrix = np.zeros((n_nodes, n_nodes), dtype="int64")
graph_matrix[0, node_offsets[0]:node_offsets[1]] = df["n_shifts"].to_numpy()
#graph_matrix[node_offsets[1]:node_offsets[2], -1] = 3

shiftlead_vector = np.zeros(df.shape[0]*len(SHIFTS)*len(DAYS))
stretch_factor = len(SHIFTS)*len(DAYS)

for s in shift_type_dict:
    df_sl = None
    for st in shift_type_dict[s]:
        if df_sl is None:
            df_sl = df["sl_" + st].to_numpy().copy()
        else:
            df_sl |= df["sl_" + st].to_numpy()
    if df_sl is None:
        df_sl = np.array(df.shape[0] * [False])
    keysplit = s.split('_')
    column_index = DAYS.index(keysplit[0]) * 3 + SHIFTS.index(keysplit[1])
    shiftlead_vector[column_index::stretch_factor] = df_sl
    graph_matrix[node_offsets[1]:node_offsets[2],node_offsets[2]+column_index] = shiftlead_vector

for index, row in df.iterrows():
    avai = row.to_numpy()[3:3+(len(SHIFTS)*len(DAYS))]
    start_col = node_offsets[1]+(index*len(SHIFTS)*len(DAYS))
    end_col = node_offsets[1]+((index+1)*len(SHIFTS)*len(DAYS))
    graph_matrix[node_offsets[0]+index, start_col:end_col] = avai
    #graph_matrix[start_col:end_col, node_offsets[2]:node_offsets[3]] = np.identity(len(SHIFTS)*len(DAYS))
    graph_matrix[start_col:end_col, node_offsets[3]:node_offsets[4]] = np.identity(len(SHIFTS)*len(DAYS))
    graph_matrix[start_col:end_col, node_offsets[4]:node_offsets[5]] = np.identity(len(SHIFTS)*len(DAYS))


graph_matrix[node_offsets[4]:node_offsets[5], 0] = 1
graph_matrix[0,node_offsets[5]] = 1000

edge_costs={}
for i in range(n_nodes):
    for j in range(n_nodes):
        if j == 0 and node_offsets[4] <= i < node_offsets[5]:
            edge_costs[i,j] = -1
        else:
            edge_costs[i,j] = 0

node_demands=np.zeros((n_nodes),dtype="int16")
for i in range(n_nodes):
    if i == 0:
        node_demands[i] = -1000
    elif node_offsets[2] <= i < node_offsets[4]:
        node_demands[i] = 1
    elif i == node_offsets[5]:
        node_demands[i] = 1000 - (2*len(SHIFTS)*len(DAYS))
    else:
        node_demands[i] = 0

# disable shifts
def disable_shift(i):
    node_demands[node_offsets[2]+i] = 0
    node_demands[node_offsets[3]+i] = 0
    node_demands[0] += 2
    graph_matrix[:,node_offsets[4]+i] = 0

for s in shift_type_dict:
    keysplit = s.split('_')
    column_index = DAYS.index(keysplit[0]) * 3 + SHIFTS.index(keysplit[1])
    if len(shift_type_dict[s]) == 0:
        disable_shift(column_index)
        print("disabled shift",s)

In [ ]:
disable_shift(15)
disable_shift(20)

graph = nx.from_numpy_array(graph_matrix, create_using=nx.DiGraph())

In [ ]:
nx.set_edge_attributes(graph, edge_costs, "cost")

In [ ]:
node_demands = dict(zip(range(len(node_demands)),list(node_demands)))
nx.set_node_attributes(graph, node_demands, "demand")

In [ ]:
flow = nx.min_cost_flow(graph,capacity="weight",weight="cost")

In [ ]:
def get_matched_person(df,flow,shift,position):
    keysplit = shift.split('_')
    shift_ind = DAYS.index(keysplit[0]) * 3 + SHIFTS.index(keysplit[1])
    for i in range(df.shape[0]*len(DAYS)*len(SHIFTS)):
        if (node_offsets[position]+shift_ind) in flow[i+node_offsets[1]] and flow[i+node_offsets[1]][node_offsets[position]+shift_ind] > 0:
            return (df.iloc[i//(len(DAYS)*len(SHIFTS))]["name"])

In [ ]:
for d in DAYS:
    for s in SHIFTS:
        shift = d + "_" + s
        print(d,s)
        print(get_matched_person(df,flow,shift,2),"(lead)")
        print(get_matched_person(df,flow,shift,3))
        print(get_matched_person(df,flow,shift,4))
        print()
            